<a href="https://colab.research.google.com/github/fjadidi2001/AD_Prediction/blob/main/Mel_specto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Extraction and labeling

In [3]:
import zipfile
import os
import pandas as pd
from glob import glob

# Step 1: Extract the zip file
zip_path = '/content/drive/MyDrive/AD/Mel_Spectrograms.zip'
extract_path = '/content'

# Extract the zip file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Extraction completed!")
print("Contents of /content:")
print(os.listdir('/content'))

# Check if Mel_Spectrograms folder exists
if 'Mel_Spectrograms' in os.listdir('/content'):
    print("\nMel_Spectrograms folder found!")
    mel_path = '/content/Mel_Spectrograms'
    print("Contents of Mel_Spectrograms:")
    print(os.listdir(mel_path))

    # Check subdirectories
    subdirs = os.listdir(mel_path)
    print(f"\nSubdirectories: {subdirs}")

    for subdir in subdirs:
        subdir_path = os.path.join(mel_path, subdir)
        if os.path.isdir(subdir_path):
            files = os.listdir(subdir_path)
            print(f"Files in {subdir}: {len(files)} files")
            if len(files) > 0:
                print(f"Sample files: {files[:5]}")
else:
    print("Mel_Spectrograms folder NOT found!")
    # List all contents to see what was actually extracted
    for root, dirs, files in os.walk('/content'):
        level = root.replace('/content', '').count(os.sep)
        indent = ' ' * 2 * level
        print(f'{indent}{os.path.basename(root)}/')
        subindent = ' ' * 2 * (level + 1)
        for file in files:
            print(f'{subindent}{file}')

# Now try to find the correct paths
base_path = '/content/Mel_Spectrograms'
if not os.path.exists(base_path):
    # Try to find where the folder actually is
    for root, dirs, files in os.walk('/content'):
        if 'Mel_Spectrograms' in dirs:
            base_path = os.path.join(root, 'Mel_Spectrograms')
            break

print(f"\nUsing base path: {base_path}")

# Check if AD and CN directories exist
ad_path = os.path.join(base_path, 'AD')
cn_path = os.path.join(base_path, 'CN')

print(f"AD path exists: {os.path.exists(ad_path)}")
print(f"CN path exists: {os.path.exists(cn_path)}")

if os.path.exists(ad_path):
    print(f"AD directory contents: {os.listdir(ad_path)[:10]}")  # Show first 10 files
if os.path.exists(cn_path):
    print(f"CN directory contents: {os.listdir(cn_path)[:10]}")  # Show first 10 files

# Try different file extensions
extensions = ['*.png', '*.jpg', '*.jpeg', '*.npy', '*.npz']
ad_files = []
cn_files = []

for ext in extensions:
    if os.path.exists(ad_path):
        ad_files = glob(os.path.join(ad_path, ext))
        if ad_files:
            print(f"Found {len(ad_files)} AD files with extension {ext}")
            break

for ext in extensions:
    if os.path.exists(cn_path):
        cn_files = glob(os.path.join(cn_path, ext))
        if cn_files:
            print(f"Found {len(cn_files)} CN files with extension {ext}")
            break

# Create labels
labels = [1] * len(ad_files) + [0] * len(cn_files)
file_paths = ad_files + cn_files

print(f"\nTotal AD files: {len(ad_files)}")
print(f"Total CN files: {len(cn_files)}")

# Create DataFrame
df = pd.DataFrame({
    'file_path': file_paths,
    'label': labels
})

print(f"\nDataset shape: {df.shape}")
print("\nFirst few entries:")
print(df.head())

# Save to CSV
if not df.empty:
    df.to_csv('/content/alzheimers_dataset.csv', index=False)
    print("\nDataset saved to /content/alzheimers_dataset.csv")
else:
    print("\nNo files found. Please check the zip file contents.")

Extraction completed!
Contents of /content:
['.config', 'alzheimers_dataset.csv', 'drive', 'Mel_Spectrograms', 'sample_data']

Mel_Spectrograms folder found!
Contents of Mel_Spectrograms:
['AD', 'CN']

Subdirectories: ['AD', 'CN']
Files in AD: 87 files
Sample files: ['adrso063.npy', 'adrso192.npy', 'adrso144.npy', 'adrso110.npy', 'adrso122.npy']
Files in CN: 79 files
Sample files: ['adrso316.npy', 'adrso012.npy', 'adrso310.npy', 'adrso160.npy', 'adrso309.npy']

Using base path: /content/Mel_Spectrograms
AD path exists: True
CN path exists: True
AD directory contents: ['adrso063.npy', 'adrso192.npy', 'adrso144.npy', 'adrso110.npy', 'adrso122.npy', 'adrso060.npy', 'adrso039.npy', 'adrso106.npy', 'adrso112.npy', 'adrso250.npy']
CN directory contents: ['adrso316.npy', 'adrso012.npy', 'adrso310.npy', 'adrso160.npy', 'adrso309.npy', 'adrso292.npy', 'adrso014.npy', 'adrso315.npy', 'adrso157.npy', 'adrso022.npy']
Found 87 AD files with extension *.npy
Found 79 CN files with extension *.npy

To